# Google Trends in AQL

In this notebook, we inspect the most prominent queries with respect to certain time windows and compare them to the most frequent queries from google in that same time window.
### 1. Annual Analysis
First, we look at the most frequent queries in a year. We examine the years from 1999 up to 2022.

Let's load the annual top 25 queries from google:

In [ ]:
import pandas as pd

annual_google_trends = pd.read_csv("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/google_trends/google_trends_total.csv")

# print(annual_google_trends['year'].unique())
# print(annual_google_trends.columns)
# for i in range(2004, 2021):
    # print(i, annual_google_trends[annual_google_trends['year'] == i])
# print(annual_google_trends)

2004           query  score  year
450        free    100  2004
451      lyrics     66  2004
452     windows     47  2004
453       music     35  2004
454       yahoo     35  2004
455         dvd     35  2004
456         mp3     34  2004
457       games     33  2004
458         web     26  2004
459         msn     25  2004
460        news     24  2004
461        ebay     21  2004
462        test     21  2004
463       linux     20  2004
464     weather     18  2004
465        love     17  2004
466        sony     17  2004
467       crack     17  2004
468        java     17  2004
469       flash     17  2004
470        chat     16  2004
471     hotmail     16  2004
472   microsoft     16  2004
473  dictionary     14  2004
474      cheats     14  2004
2005           query  score  year
425        free    100  2005
426    download     77  2005
427      lyrics     73  2005
428       yahoo     43  2005
429         msn     36  2005
430       games     35  2005
431       music     34  2005
432 

Now, we load the annual top 25 queries, the annual top 25 english queries and the annual top 25 google queries from the the aql:

In [49]:
annual_aql_trends = pd.read_parquet("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aql-get-annual-top-queries-special")
annual_aql_trends_eng = pd.read_parquet("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aql-get-annual-top-queries-english")
annual_aql_trends_google = pd.read_parquet("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/analysis_data/analysis/aql-get-annual-top-queries-google")


annual_aql_trends.rename(columns={'serp_query_text_url': 'query', 'count()': 'score'}, inplace=True)
annual_aql_trends_eng.rename(columns={'serp_query_text_url': 'query', 'count()': 'score'}, inplace=True)
annual_aql_trends_google.rename(columns={'serp_query_text_url': 'query', 'count()': 'score'}, inplace=True)
# print(annual_aql_trends.columns)
print(annual_aql_trends['score'].sum())
aql_top_queries = {}
aql_top_queries.update({'aql_google': annual_aql_trends_google})
aql_top_queries.update({'aql_english': annual_aql_trends_eng})
aql_top_queries.update({'aql_native': annual_aql_trends})


83543021


Let's find out if we have intersections in the annual top 25 of both query logs:

In [50]:

# transform counts into scores by assigning a score of 100 to the most frequent query of each year and compute the ratio of the other queries
sizes = {}
for key, data in aql_top_queries.items():
    print(f"AQL {key.upper()}:")
    years = set(list(data['year']))
    sizes.update({f"{key}_count": [data['score'].sum(), 100*data['score'].sum()/346310968]})
    for year in years:
        # print(year)
        max_count = data[data['year'] == year]['score'].max()
        data.loc[data['year'] == year, 'score'] = round(data.loc[data['year'] == year, 'score'] / max_count * 100)
        # print(year)

    years = set(list(data['year']))
    for year in years:
        if year >= 2004:
            aql_queries = set(data[data['year'] == year]['query'].reset_index(drop=True))
            google_queries = set(annual_google_trends[annual_google_trends['year'] == year]['query'].reset_index(drop=True))
            # google_queries_set = set(google_queries)
            # find matching queries
            matches = google_queries.intersection(aql_queries)
            # print the matching queries
            print(f"{year}: {matches}")

for key,value in sizes.items():
    print(f"{key}: {value}")    


AQL AQL_GOOGLE:
2004: set()
2005: set()
2006: set()
2007: set()
2008: set()
2009: set()
2010: set()
2011: set()
2012: {'youtube', 'facebook', 'google'}
2013: {'youtube', 'yahoo', 'facebook', 'google'}
2014: set()
2015: set()
2016: set()
2017: set()
2018: set()
2019: set()
2020: set()
2021: set()
2022: set()
AQL AQL_ENGLISH:
2004: set()
2005: set()
2006: set()
2007: set()
2008: set()
2009: set()
2010: set()
2011: set()
2012: set()
2013: set()
2014: set()
2015: set()
2016: set()
2017: set()
2018: set()
2019: set()
2020: set()
2021: set()
2022: set()
AQL AQL_NATIVE:
2004: {'free'}
2005: set()
2006: set()
2007: {'video'}
2008: set()
2009: set()
2010: set()
2011: {'you'}
2012: {'google'}
2013: set()
2014: set()
2015: set()
2016: set()
2017: set()
2018: set()
2019: set()
2020: set()
2021: set()
2022: set()
aql_google_count: [16596750, 4.792441341332279]
aql_english_count: [2412914, 0.6967477853603528]
aql_native_count: [83543021, 24.123700581149368]


As we can see, there are only few intersections in the annual top 25 queries of google and the aql queries. 

### 2. Monthly Analysis

We have loaded the top 25 queries of each month from 2005 until 2022 from google. In this analysis we want to compare the actual courses of query frequencies from the AQL and Google. For this, we first need to make a selection which queries we want for that analysis. We take the top 25 queries of the most frequent monthly queries from google. This set is obtained by applying [reciprocal rank fusion](https://dl.acm.org/doi/abs/10.1145/1571941.1572114) to the list of the top 25 monthly queries from 2005 - 2022 by google. 
First, we load google trends data:  

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

# this function groups the data by query and computes the reciprocal rank fusion score for each query 
def get_rrf_score(data: pd.DataFrame) -> pd.DataFrame:
    data = data.groupby('query').agg(rrf_score=pd.NamedAgg(column='rank', aggfunc=lambda x: np.sum(1/(60+x)))).reset_index()
    return data.sort_values('rrf_score', ascending=False)

base_path = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/thesis-schneg/google_trends/monthly")

data = pd.DataFrame()
for path in base_path.iterdir():
    if path.is_file():
        df = pd.read_csv(path)
        data = pd.concat([data, df])

print(data)
print(data.shape)
print(list(get_rrf_score(data)['query'].head(100)))


               query  score  rank
0           facebook    100     1
1             google     51     2
2            youtube     49     3
3                you     29     4
4               news     17     5
...              ...    ...   ...
5695  facebook login      8    21
5696              as      8    22
5697            ebay      8    23
5698              dr      8    24
5699           apple      8    25

[5700 rows x 3 columns]
(5700, 3)
['google', 'yahoo', 'weather', 'youtube', 'hotmail', 'facebook', 'gmail', 'news', 'you', 'ebay', 'amazon', 'games', 'free', 'twitter', 'translate', 'mp3', 'maps', 'msn', 'fb', 'mail', 'instagram', 'map', 'face', 'video', 'juegos', 'craigslist', 'facebook login', 'lyrics', 'game', 'traductor', 'you tube', 'as', 'whatsapp', 'videos', 'wikipedia', 'yahoo mail', 'myspace', 'tiempo', 'samsung', 'bbc', 'meteo', 'web', 'music', 'nokia', 'погода', 'clima', 'chat', 'sony', 'download', 'go', 'google translate', 'wiki', 'netflix', 'hot', 'dictionary', 'messenger

Now, we compare the time series of the top 25 monthly google queries over a large time span to the respective frequencies of those queries in the AQL over the same time span.